### Create a new BQ dataset to store all of your modeled tables. The dataset should be named datamart.


In [3]:
dataset_id = "datamart"

In [4]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'still-bank-302722:datamart' successfully created.


### Create your modeled tables by following our BigQuery design guidelines (refer to slide 3 from lecture slides for details)

● split staging tables that contain more than one entity into separate tables.

● join staging tables that store different attributes belonging to the same entity.

● union staging tables that store distinct records belonging to the same entity.

● identify a candidate primary key (PK) for each modeled table.

● check for records with the same primary key and remove unwanted ones.

● identify parent-child relationships between tables.

● check for referential integrity violations between parent-child tables.

● remove any child records which violate referential integrity.


Convert string nulls to int nulls (birthYear and deathYear)

In [8]:
%%bigquery
UPDATE IMDB_staging.name_basics nb
SET nb.birthYear = '0'
WHERE birthYear = "\\N"

""


In [13]:
%%bigquery
UPDATE IMDB_staging.name_basics nb
SET nb.deathYear = '0'
WHERE deathYear = "\\N"

""


### Create Name_basics table

In [15]:
%%bigquery
CREATE TABLE datamart.Name_basics as
SELECT nconst, primaryName, CAST(birthYear as INT64) as birthYear, CAST(deathYear as INT64) as deathYear, primaryProfession, knownForTitles 
FROM IMDB_staging.name_basics

""


Checking primary key (nconst) in Name_basics table

In [12]:
%%bigquery
SELECT count(*) as totalRecords, count(nconst) as primaryKeyCount
FROM datamart.Name_basics

,totalRecords,primaryKeyCount
0,10857084,10857084


Convert string nulls to int nulls (startYear, endYear, runtimeMinutes)

In [5]:
%%bigquery
UPDATE IMDB_staging.title_basics
SET startYear = '0'
WHERE startYear = '\\N'

""


In [6]:
%%bigquery
UPDATE IMDB_staging.title_basics
SET endYear = '0'
WHERE endYear = '\\N'

""


In [7]:
%%bigquery
UPDATE IMDB_staging.title_basics
SET runtimeMinutes = '0'
WHERE runtimeMinutes = '\\N'

""


### Create Title_basics table

In [13]:
%%bigquery
CREATE TABLE datamart.Title_basics as
SELECT tconst as titleId, titleType, primaryTitle, originalTitle, CAST(startYear as INT64) as startYear, CAST(endYear as INT64) as endYear , CAST(runtimeMinutes as INT64) as runtimeMinutes, genres
FROM IMDB_staging.title_basics

""


Checking primary key (titleId) in Title_basics table

In [11]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyCount
FROM datamart.Title_basics

,totalRecords,primaryKeyCount
0,7806188,7806188


### Creating Title_principals table

In [14]:
%%bigquery
CREATE TABLE datamart.Title_principals as
SELECT tconst as titleId, ordering, nconst, category, job, characters
FROM IMDB_staging.title_principals

""


Checking primary key (titleId, ordering) and the foreign keys(nconst, titleId) in Title_basics table

In [13]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as titleIdPrimaryKeyCount, count(ordering) as orderingPrimaryKeyCount, count(nconst) as foreignKeyCount
FROM datamart.Title_principals

,totalRecords,titleIdPrimaryKeyCount,orderingPrimaryKeyCount,foreignKeyCount
0,44159688,44159688,44159688,44159688


Convert string nulls to int nulls (seasonNumber and episodeNumber)

In [17]:
%%bigquery
UPDATE IMDB_staging.title_episode
SET seasonNumber = '0'
WHERE seasonNumber = '\\N'

""


In [18]:
%%bigquery
UPDATE IMDB_staging.title_episode
SET episodeNumber = '0'
WHERE episodeNumber = '\\N'

""


### Creating Title_episode table

In [20]:
%%bigquery
CREATE TABLE datamart.Title_episode as
SELECT tconst as epId, parentTconst as titleId, CAST(seasonNumber as INT64) as seasonNumber, CAST(episodeNumber as INT64) as episodeNumber
FROM IMDB_staging.title_episode

""


Checking primary key (titleId) and foreign Key (epid) in Title_episode table

In [10]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyCount, count(epid) as foreignKeyCount
FROM datamart.Title_episode

,totalRecords,primaryKeyCount,foreignKeyCount
0,5683026,5683026,5683026


### Creating Title_crew table

In [21]:
%%bigquery
CREATE TABLE datamart.Title_crew as
SELECT tconst as titleId, directors, writers
FROM IMDB_staging.title_crew

""


Checking primary key/foreign key (titleId) in Title_crew table

In [15]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyForeignKeyCount
FROM datamart.Title_crew

,totalRecords,primaryKeyForeignKeyCount
0,7806188,7806188


Convert string '0' and '1' values to boolean values (isOriginalTitle)

In [13]:
%%bigquery
UPDATE IMDB_staging.title_akas
SET isOriginalTitle = 'TRUE'
WHERE isOriginalTitle = '1'

""


In [14]:
%%bigquery
UPDATE IMDB_staging.title_akas
SET isOriginalTitle = 'FALSE'
WHERE isOriginalTitle = '0'

""


In [15]:
%%bigquery
UPDATE IMDB_staging.title_akas
SET isOriginalTitle = 'NULL'
WHERE isOriginalTitle = '\\N'

""


### Creating Title_akas table

In [16]:
%%bigquery
CREATE TABLE datamart.Title_akas as
SELECT titleId, ordering, region, language, types, attributes, SAFE_CAST(isOriginalTitle as BOOL) as isOriginalTitle
FROM IMDB_staging.title_akas

""


Checking primary key/foreign key (titleId) and primaryKey(ordering) in Title_akas table

In [16]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyForeignKeyCount, count(ordering) as primaryKeyCount
FROM datamart.Title_akas

,totalRecords,primaryKeyForeignKeyCount,primaryKeyCount
0,25920383,25920383,25920383


### Creating Title_ratings table

In [1]:
%%bigquery
CREATE TABLE datamart.Title_ratings as
SELECT tconst as titleId, averageRating, numVotes
FROM IMDB_staging.title_ratings

""


Checking primary key/foreign key (titleId) in Title_ratings table

In [17]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyForeignKeyCount
FROM datamart.Title_ratings

,totalRecords,primaryKeyForeignKeyCount
0,1139872,1139872


### Creating Bollywood table by joining the title_basics table and creating the tconst (titleId) column

In [6]:
%%bigquery
CREATE TABLE datamart.Bollywood as
SELECT tb.tconst as titleId, b.*
FROM Bollywood_staging.bollywood b join IMDB_staging.title_basics tb on b.Title = tb.originalTitle

""


Checking primary key/foreign key (titleId) in Bollywood table

In [18]:
%%bigquery
SELECT count(*) as totalRecords, count(titleId) as primaryKeyForeignKeyCount
FROM datamart.Bollywood

,totalRecords,primaryKeyForeignKeyCount
0,37719,37719


### Creating Bollywood_actress table

In [2]:
%%bigquery
CREATE TABLE datamart.Bollywood_actress as
SELECT Name as name, Height_in_cm_ as heightInCm, Debut_aslead_role as debutAsLeadRole
FROM Bollywood_staging.bollywood_actress

""


Checking primary key (name) in Bollywood_actress table

In [19]:
%%bigquery
SELECT count(*) as totalRecords, count(name) as primaryKeyCount
FROM datamart.Bollywood_actress

,totalRecords,primaryKeyCount
0,134,134


### Creating Bollywood_actors table

In [3]:
%%bigquery
CREATE TABLE datamart.Bollywood_actors as
SELECT Name as name, Height_in_cm_ as heightInCm
FROM Bollywood_staging.bollywood_actors

""


Checking primary key (name) in Bollywood_actors table

In [21]:
%%bigquery
SELECT count(*) as totalRecords, count(name) as primaryKeyCount
FROM datamart.Bollywood_actors

,totalRecords,primaryKeyCount
0,139,139


### Continue to explore your data by writing SQL queries on your modeled tables:

● Come up with 5 new queries, at least 3 of which should contain a subquery and
at least 2 of which should contain an aggregation.

● Each query should also include at least 2 clauses from this list: JOIN, WHERE,
GROUP BY, HAVING, ORDER BY, LIMIT.

● Precede each query with a Markdown comment that describes its function

### Who are the top 10 modern Bollywood directors based on the amount of movies they made from 1995 - 2021 (excluding unknowns)

In [45]:
%%bigquery
SELECT b.director, count(b.title) as movieCount
FROM datamart.Bollywood b
WHERE EXISTS(SELECT b2.year FROM datamart.Bollywood b2 WHERE b2.year > 1995 and b2.year < 2021 and b2.director != "Unknown" and b.director != "Unknown" and b.year = b2.year)
GROUP BY b.director
ORDER BY movieCount DESC
Limit 10

,director,movieCount
0,Vikramaditya Motwane,391
1,Bhushan Patel,357
2,Anees Bazmee,288
3,Vikram Bhatt,285
4,Kabir Khan,230
5,Mukul Abhyankar,224
6,Jeeva,201
7,Paresh Vinodray Savani,200
8,Karan Malhotra,190
9,Saawan Kumar,179


### Who are the top 10 modern IMDB directors based on the amount of movies they made from 1995 - 2021 (excluding None type)

In [54]:
%%bigquery
SELECT n.primaryName, count(tb.primaryTitle) as movieCount
FROM datamart.Title_basics tb
LEFT JOIN datamart.Title_crew c on tb.titleId = c.titleId
LEFT JOIN datamart.Name_basics n on c.directors = n.nconst
WHERE n.primaryName != "None" and EXISTS(SELECT tb.startYear FROM datamart.Title_basics tb2 WHERE tb2.startYear > 1995 and tb2.startYear < 2021 and tb.startYear = tb2.startYear)
GROUP BY n.primaryName
ORDER BY movieCount DESC
Limit 10

,primaryName,movieCount
0,Kevin McCarthy,5194
1,Khalil Herekar,3200
2,Pawan Kumar,3108
3,Chuck O'Neil,2859
4,Kaushik Ghatak,2808
5,Mark Corwin,2737
6,Nikos Foskolos,2496
7,Lewis Brindley,2343
8,Shashank Bali,2333
9,Ramji,2066


### IMDB Top rated movies from top director 

In [29]:
%%bigquery
SELECT n.primaryName, tb.primaryTitle, (SELECT MAX(tb2.averageRating) FROM datamart.Title_ratings tb2) as rating
FROM datamart.Title_basics tb
LEFT JOIN datamart.Title_crew c on tb.titleId = c.titleId
LEFT JOIN datamart.Name_basics n on c.directors = n.nconst
WHERE n.primaryName = "Kevin McCarthy" and tb.titleType = "movie"

,primaryName,primaryTitle,rating
0,Kevin McCarthy,The Rouge Shoes,10.0


### Bollywood top rated movies from top director 

In [30]:
%%bigquery
SELECT DISTINCT b.director, b.title, (SELECT MAX(tb2.averageRating) FROM datamart.Title_ratings tb2) as rating
FROM datamart.Bollywood b
JOIN datamart.Title_basics tb on b.titleId = tb.titleId
JOIN datamart.Title_ratings tr on tb.titleID = tr.titleId
WHERE b.director = "Vikramaditya Motwane" and tb.titleType = "movie" 
Limit 10

,director,title,rating
0,Vikramaditya Motwane,Trapped,10.0
1,Vikramaditya Motwane,Udaan,10.0
2,Vikramaditya Motwane,Lootera,10.0
3,Vikramaditya Motwane,Bhavesh Joshi Superhero,10.0


###  What are the top 10 languages of the top grossing Bollywood movies? 

In [37]:
%%bigquery 
SELECT b.title, ta.language, (SELECT MAX(b2.Highest_Grosser_By_Year_in_crores_) FROM datamart.Bollywood b2) as GrossAmount
FROM datamart.Bollywood b
JOIN datamart.Title_basics tb on b.titleId = tb.titleId
JOIN datamart.Title_akas ta on tb.titleId = ta.titleId
WHERE ta.language != "\\N"
LIMIT 10

,title,language,GrossAmount
0,Om Jai Jagadish,fr,2024.0
1,War,tr,2024.0
2,Alone,en,2024.0
3,Ghar Aaya Mera Pardesi,en,2024.0
4,Page 3,hi,2024.0
5,Goliyon Ki Rasleela Ram-Leela,en,2024.0
6,The Lunchbox,tr,2024.0
7,Bambai Ki Sethani,en,2024.0
8,Jallaad,en,2024.0
9,Baazigar,en,2024.0


### Create data visualizations:
● Create a new BQ dataset for storing your reporting views. Name the dataset reports.

● Choose 2 of your most interesting queries from the previous section.

● Create a view for each query in the reports dataset and assign the view a descriptive name (e.g.Highest_Nominated_Movies).

● Open Data Studio

● In Data Studio, create a Data Source (using the + Create button) that accesses the views. You’ll need one Data Source per view.

● Create a chart that visualizes the data in a compelling way.

● Add both charts to a single Data Studio report (aka dashboard).

● Download your dashboard as a pdf and name it dashboard-v1.pdf

In [38]:
dataset_id2 = "reports"

In [39]:
!bq --location=US mk --dataset {dataset_id2}

Dataset 'still-bank-302722:reports' successfully created.


In [43]:
%%bigquery 
CREATE VIEW reports.Top_10_Bollywood_Directors AS
    SELECT b.director, count(b.title) as movieCount
    FROM datamart.Bollywood b
    WHERE EXISTS(SELECT b2.year FROM datamart.Bollywood b2 WHERE b2.year > 1995 and b2.year < 2021 and b2.director != "Unknown" and b.director != "Unknown" and b.year = b2.year)
    GROUP BY b.director
    ORDER BY movieCount DESC
    Limit 10

""


In [44]:
%%bigquery 
CREATE VIEW reports.Top_10_IMDB_Directors AS
    SELECT n.primaryName, count(tb.primaryTitle) as movieCount
    FROM datamart.Title_basics tb
    LEFT JOIN datamart.Title_crew c on tb.titleId = c.titleId
    LEFT JOIN datamart.Name_basics n on c.directors = n.nconst
    WHERE n.primaryName != "None" and EXISTS(SELECT tb.startYear FROM datamart.Title_basics tb2 WHERE tb2.startYear > 1995 and tb2.startYear < 2021 and tb.startYear = tb2.startYear)
    GROUP BY n.primaryName
    ORDER BY movieCount DESC
    Limit 10

""
